In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [2]:
# os.rmdir('/kaggle/working/mainfolder')
for root, folders, filenames in os.walk('/kaggle'):
   print(root, folders,filenames)

/kaggle ['src', 'lib', 'input', 'working'] []
/kaggle/src [] ['script.ipynb']
/kaggle/lib ['kaggle'] []
/kaggle/lib/kaggle [] ['gcp.py']
/kaggle/input ['digit-recognizer'] []
/kaggle/input/digit-recognizer [] ['sample_submission.csv', 'train.csv', 'test.csv']
/kaggle/working [] ['__notebook__.ipynb']


In [3]:
import torch
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
# import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import (
    DataLoader,
    Dataset
) 
from PIL import Image
from matplotlib import pyplot as plt 
from tqdm import tqdm
import torchvision



In [4]:
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=8,
            kernel_size=3,#SAMW conv
            stride=1,
            padding=1,
        )
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=8,
            out_channels=16,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        self.fc1 = nn.Linear(16 * 7 * 7, num_classes)
#         self.fc2 = nn.Linear(16 * 7 * 7, num_classes)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
#         x= self.fc2(x)
        return x



In [5]:
class MnistData(Dataset):
    def __init__(self, csv_file, transform=None):
            self.annotations = pd.read_csv(csv_file)
    #         self.root_dir = root_dir
            self.transform = transform
    def __len__(self):
            return len(self.annotations)
    def __getitem__(self, index):
            image =  self.annotations.values[index,1:].reshape(28,28,-1).astype('float32')/255.0

            y_label = torch.tensor(int(self.annotations.iloc[index, 0]))

            if self.transform:
                image = self.transform(image)

            return (image, y_label)

    

In [6]:
class pretest(Dataset):
    def __init__(self, csv_file, transform=None):
            self.annotations = pd.read_csv(csv_file)
    #         self.root_dir = root_dir
            self.transform = transform
    def __len__(self):
            return len(self.annotations)
    def __getitem__(self, index):
            image =  self.annotations.values[index,:].reshape(28,28,-1).astype('float32')/255.0

#             y_label = torch.tensor(int(self.annotations.iloc[index, 0]))

            if self.transform:
                image = self.transform(image)

            return (image)

In [7]:
class test(Dataset):
    def __init__(self, csv_file1,csv_file2 ,transform=None):
            self.annotations1 = pd.read_csv(csv_file1)
            self.annotations2 = pd.read_csv(csv_file2)

    #         self.root_dir = root_dir
            self.transform = transform
    def __len__(self):
            return len(self.annotations1)
    def __getitem__(self, index):
            image =  self.annotations1.values[index,:].reshape(28,28,-1).astype('float32')/255.0

            y_label = torch.tensor(int(self.annotations2.iloc[index, 1]))

            if self.transform:
                image = self.transform(image)

            return (image,y_label)

In [8]:
def showimage(index):
    dataload = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
    row1 = dataload.values[index,1:].reshape(28,28,-1).astype('float32')/255.0
    
    # row1
    print(row1.shape)
    # image = Image.fromarray(row1)
#     plt.imshow(row1)
showimage(4)



(28, 28, 1)


In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [10]:
in_channels = 1
num_classes = 10
learning_rate = 3e-4 # karpathy's constant
batch_size = 64
num_epochs = 15

In [11]:
# Load Data
traindataset = MnistData(
    csv_file="/kaggle/input/digit-recognizer/train.csv",
    transform=transforms.ToTensor(),
)
testdataset = pretest(
    csv_file="/kaggle/input/digit-recognizer/test.csv",
    transform=transforms.ToTensor(),
) 


In [12]:
train_loader = DataLoader(dataset=traindataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=testdataset,shuffle=False)


In [13]:
# for x,y in train_loader:
#     print(x.shape)
#     x = x.permute(0,3,2,1)
#     print(x.shape)
#     break

In [14]:
model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)


In [15]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [16]:
for epoch in range(num_epochs):
    losses = []

    for batch_idx, (data, targets) in enumerate(tqdm(train_loader)):
        # Get data to cuda if possible
#         print(data.shape)

#         break

        data = data.to(device=device)
        targets = targets.to(device=device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())
        

        # backward
        optimizer.zero_grad()
        loss.backward()

        # gradient descent or adam step
        optimizer.step()
    print(f"Cost at epoch {epoch} is {sum(losses)/len(losses)}")

        
        


100%|██████████| 657/657 [00:05<00:00, 112.04it/s]


Cost at epoch 0 is 0.7901335617518861


100%|██████████| 657/657 [00:04<00:00, 132.32it/s]


Cost at epoch 1 is 0.2358642012969544


100%|██████████| 657/657 [00:04<00:00, 137.21it/s]


...

Cost at epoch 12 is 0.059682316786809464


100%|██████████| 657/657 [00:04<00:00, 132.00it/s]


Cost at epoch 13 is 0.056922401574793266


100%|██████████| 657/657 [00:04<00:00, 136.97it/s]

Cost at epoch 14 is 0.05330674409124961


In [17]:
# print(losses)

In [18]:
# df = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')
# image =  df.values[1,:].reshape(28,28,-1).astype('float32')/255.0
# image[:,:,1]
# print(image.shape)
sample = {'ImageId':[],
          'Label': []
            }
model.eval()
index =1
with torch.no_grad():
    for x in test_loader:
#         x = x.permute(0,3,2,1)
        x = x.to(device=device)
        scores = model(x)
#         print(scores.shape)
#         print(scores)
        _, predictions = scores.max(1)
#         print(predictions)
        sample['ImageId'].append(index)
        sample['Label'].append(int(predictions))
        index = index+1
#         print(sample)
df = pd.DataFrame(sample)  

    
    
    

In [19]:
df.to_csv(os.path.join('/kaggle/working/'+'output.csv'),index = False)


In [20]:
testfinaldataset = test(
    csv_file1="/kaggle/input/digit-recognizer/test.csv",

    csv_file2="/kaggle/working/output.csv",
    transform=transforms.ToTensor(),
)
testfinal_loader = DataLoader(dataset=testfinaldataset,batch_size=batch_size,shuffle=True)


In [21]:
# Check accuracy on training & test to see how good our model
def check_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
        for x, y in loader:
            x = x.to(device=device)
            y = y.to(device=device)

            scores = model(x)
            _, predictions = scores.max(1)
            num_correct += (predictions == y).sum()
            num_samples += predictions.size(0)

    model.train()
    return num_correct / num_samples


print(f"Accuracy on training set: {check_accuracy(train_loader, model)*100:.2f}")
print(f"Accuracy on test set: {check_accuracy(testfinal_loader, model)*100:.2f}")

Accuracy on training set: 98.20
Accuracy on test set: 100.00
